Pablo Marchesi

October 2024

Mail: pablomarchesiselma@gmail.com

LinkedIn: https://www.linkedin.com/in/pablo-marchesi-010383199/

For a detailed explanation, please visit:

In [2]:
import numpy as np 
import plotly.express as px 
import plotly.graph_objects as go
from numpy.random import normal
from scipy.stats import norm, gaussian_kde
from plotly.subplots import make_subplots
import pandas as pd 


## Heston Model

\begin{equation*}
dS_t = \mu S_t \, dt + \sqrt{\nu_t} S_t \, dW_t^S 
\end{equation*}

\begin{equation*}
d\nu_t = \kappa (\theta - \nu_t) \, dt + \xi \sqrt{\nu_t} \, dW_t^\nu 
\end{equation*}

\begin{equation*}
\mathbb{E}[dW_t^S \, dW_t^\nu] = \rho \, dt 
\end{equation*}

Where:
- $ S_t $ is the stock price at time $ t $,
- $ \nu_t $ is the variance of the asset price at time $ t $,
- $ \mu $ is the drift rate of the asset price,
- $ \kappa $ is the rate of mean reversion of the variance,
- $ \theta $ is the long-run mean of the variance,
- $ \xi $ is the volatility of volatility,
- $ \rho $ is the correlation between the Wiener processes $ W_t^S $ and $ W_t^\nu $,
- $ W_t^S $ and $ W_t^\nu $ are standard Brownian motions.

In [3]:
# Heston Model 
def Heston(S0, T, r, kappa, nu0, theta, xi, rho, paths):

    steps = round(252*T)
    dt = 1/steps
    nu = np.zeros((steps+1, paths))
    S = np.zeros((steps+1, paths))
    nu[0] = nu0; S[0] = S0

    for t in range(1, 1+steps):
        dW_S = normal(0, np.sqrt(dt), paths)
        Z = normal(0, np.sqrt(dt), paths)
        dW_nu = rho * dW_S + np.sqrt(1 - rho**2) * Z

        nu[t] = nu[t-1] - kappa*(nu[t-1] - theta)*dt + xi*np.sqrt(nu[t-1])*dW_nu
        nu[t] = np.maximum(nu[t], 0)
        S[t] = S[t-1] + r*S[t-1]*dt + np.sqrt(nu[t-1])*S[t-1]*dW_S

    return S

In [4]:
# Geometric Brownian Motion
def GBM(r, sigma, S0, T, paths):

    steps = round(252*T)
    dt = 1/steps
    S = np.zeros((steps+1, paths))
    S[0] = S0

    for t in range(1,steps+1):
        dW = normal(0, np.sqrt(dt), paths)
        S[t] = S[t-1] + r * S[t-1] * dt+ sigma * S[t-1] * dW

    return S

In [5]:
# Geometric Brownian Motion PDF (log prices)
def GBMpdf(x, S0, r, sigma, T):

    loc = np.log(S0) + (r - 0.5 * sigma**2)*T
    scale = sigma * np.sqrt(T)
    pdf = norm.pdf(x, loc, scale)
    
    return pdf

# Heston PDF (log_prices)
def HestonPDF(x, S_T):
    kde = gaussian_kde(np.log(S_T))
    pdf = kde(x)
    return pdf

In [6]:
# Put option price (monte carlo approach)
def put_option_price(S_T, K, r, T):
    payoffs = np.maximum(K - S_T, 0)
    return np.mean(payoffs*np.exp(-r*T))

# Put option price (Black-Scholes approach)
def bs_put_price(S0, K, T, r, sigma):
    d1 = (np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))
    d2 = d1 - sigma * np.sqrt(T)
    return K * np.exp(-r * T) * norm.cdf(-d2) - S0 * norm.cdf(-d1)


In [7]:
# Implied volatility calculation
def implied_vol(S0, K, T, r, option_price, tol=1e-5, max_iter=100):
    sigma = 0.2
    
    for _ in range(max_iter):
        price = bs_put_price(S0, K, T, r, sigma)
        vega = S0 * norm.pdf((np.log(S0 / K) + (r + 0.5 * sigma**2) * T) / (sigma * np.sqrt(T))) * np.sqrt(T)
        diff = price - option_price

        if abs(diff) < tol:
            return sigma
        sigma -= diff / vega

    return sigma

In [16]:
# -------SIMULATION OF GBM AND HESTON MODEL------- 

S0 = 100        # Initial stock price
T = 1           # Time to maturity (years)
r = 0.05        # Risk-free rate
kappa = 2       # Speed of reversion
nu0 = 0.1       # Initial variance
theta = 0.09    # Long term mean
xi = 0.5        # Volatility of volatility
rho = -0.5      # Correlation between asset and volatility
paths = 100000  # Number of simulations
sigma = 0.3     # Constant volatility for GBM

# Simulation of Heston Model and GBM
S_Heston = Heston(S0, T, r, kappa, nu0, theta, xi, rho, paths)
S_GBM = GBM(r, sigma, S0, T, paths)

# Compute the Heston PDF (using a kernel density estimator)
min_val = min(np.min(np.log(S_Heston[-1])), np.min(np.log(S_GBM[-1])))
max_val = max(np.max(np.log(S_Heston[-1])), np.max(np.log(S_GBM[-1])))
x_vals = np.linspace(min_val, max_val, 5000)
pdf_heston = HestonPDF(x_vals, S_Heston[-1])

# Compute the GBM PDF (analytic solution)
pdf_gbm = GBMpdf(x_vals, S0, r, sigma, T)
df_pdf = pd.DataFrame({'Log Prices':x_vals,'Heston PDF':pdf_heston, 'GBM PDF':pdf_gbm})
df_pdf_filt = df_pdf[(df_pdf['GBM PDF'] > 1e-3) & (df_pdf['Heston PDF'] > 1e-3)]


# -------OPTION PRICING AND VOLATILITY SMILE-------

# Strikes 
strikes = np.linspace(S0*0.8, S0*1.3, 100)

# Compute the call prices for Heston Model and GBM
put_GBM = [put_option_price(S_GBM[-1], K, r, T) for K in strikes]
put_Heston = [put_option_price(S_Heston[-1], K, r, T) for K in strikes]

# Compute implied vols for Heston and GBM
implied_vols_Heston = [implied_vol(S0, K, T, r, price) for K, price in zip(strikes, put_Heston)]
implied_vols_GBM = [implied_vol(S0, K, T, r, price) for K, price in zip(strikes, put_GBM)]
df_smile = pd.DataFrame({'Strike':strikes, 'Implied Vol Heston':implied_vols_Heston, 'Implied Vol GBM': implied_vols_GBM})


# -------VOLATILITY SMILE AND PDFs PLOTS-------

fig = make_subplots(rows=1, cols=2, subplot_titles=('Volatility Smile', 'Probability Density Function'))

fig.add_trace(go.Scatter(x=df_smile['Strike'], y=df_smile['Implied Vol GBM'], 
                         mode='lines', name='GBM', line=dict(color='red')), row=1, col=1)

fig.add_trace(go.Scatter(x=df_smile['Strike'], y=df_smile['Implied Vol Heston'], 
                         mode='lines', name='Heston', line=dict(color='blue')), row=1, col=1)

fig.add_trace(go.Scatter(x=df_pdf_filt['Log Prices'], y=df_pdf_filt['GBM PDF'], 
                         mode='lines', line=dict(color='red'), showlegend=False), row=1, col=2)

fig.add_trace(go.Scatter(x=df_pdf_filt['Log Prices'], y=df_pdf_filt['Heston PDF'], 
                         mode='lines', line=dict(color='blue'), showlegend=False), row=1, col=2)

fig.update_layout(xaxis_title="Strike", yaxis_title="Implied Volatility", xaxis2_title="Log Prices", yaxis2_title="PDF")
fig.show()
